https://langchain-ai.github.io/langgraph/tutorials/reflection/reflection/

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
from langchain_openai import ChatOpenAI


In [2]:
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_fireworks import ChatFireworks

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an essay assistant tasked with writing excellent 5-paragraph essays."
            " Generate the best essay possible for the user's request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
llm = ChatOpenAI(model="gpt-3.5-turbo")

generate = prompt | llm

In [3]:
essay = ""
request = HumanMessage(
    content="Write an essay on why the little prince is relevant in modern childhood"
)
for chunk in generate.stream({"messages": [request]}):
    print(chunk.content, end="")
    essay += chunk.content

"The Little Prince" by Antoine de Saint-Exupéry is a timeless classic that continues to resonate with readers, especially in modern childhood. This enchanting tale offers valuable life lessons and insights that are still relevant today, making it a significant piece of literature for children in the contemporary world.

Firstly, one of the reasons why "The Little Prince" remains relevant in modern childhood is its exploration of the importance of imagination and creativity. In today's fast-paced and technology-driven society, children are often consumed by gadgets and electronic devices, leaving little room for imaginative play. The Little Prince's whimsical adventures and vivid imagination serve as a reminder of the power of creativity and the magic of seeing the world through a child's eyes.

Moreover, the themes of friendship and human connection depicted in the story are timeless and continue to be relevant in today's society. In an increasingly digital world, where social media of

In [4]:
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a teacher grading an essay submission. Generate critique and recommendations for the user's submission."
            " Provide detailed recommendations, including requests for length, depth, style, etc.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
reflect = reflection_prompt | llm

In [5]:
reflection = ""
for chunk in reflect.stream({"messages": [request, HumanMessage(content=essay)]}):
    print(chunk.content, end="")
    reflection += chunk.content

Your essay on why "The Little Prince" is relevant in modern childhood is insightful and well-structured. You have effectively highlighted key themes from the book and connected them to contemporary issues faced by children today. Here are some recommendations to further enhance your essay:

1. **Provide Specific Examples:** While you have touched upon important themes from the book, consider including specific examples or scenes from "The Little Prince" to support your arguments. This will help illustrate how the themes manifest in the story and make your points more compelling.

2. **Expand on the Impact:** You could delve deeper into how the lessons from "The Little Prince" can positively impact children in today's world. For instance, elaborate on how fostering imagination and creativity can benefit children's cognitive development and problem-solving skills in the digital age.

3. **Include Counterarguments:** To strengthen your essay, consider addressing potential counterarguments

In [6]:
for chunk in generate.stream(
    {"messages": [request, AIMessage(content=essay), HumanMessage(content=reflection)]}
):
    print(chunk.content, end="")

"The Little Prince" by Antoine de Saint-Exupéry continues to hold relevance in modern childhood due to its timeless themes and profound insights that resonate with young readers. This beloved tale offers valuable lessons that are essential for children growing up in today's world.

One of the key themes in "The Little Prince" is the importance of imagination and creativity. For example, when the Little Prince describes his encounters with various characters on different planets, such as the conceited man and the king, it highlights the power of imagination in understanding the complexities of human nature. In a digital age where children are often glued to screens, fostering imagination through storytelling and creative play can help enhance cognitive development and problem-solving skills.

Furthermore, the book emphasizes the significance of human connection and empathy, which are vital in today's interconnected world. The Little Prince's friendship with the fox teaches children abou

In [7]:
from typing import List, Sequence

from langgraph.graph import END, MessageGraph


async def generation_node(state: Sequence[BaseMessage]):
    return await generate.ainvoke({"messages": state})


async def reflection_node(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
    # Other messages we need to adjust
    cls_map = {"ai": HumanMessage, "human": AIMessage}
    # First message is the original user request. We hold it the same for all nodes
    translated = [messages[0]] + [
        cls_map[msg.type](content=msg.content) for msg in messages[1:]
    ]
    res = await reflect.ainvoke({"messages": translated})
    # We treat the output of this as human feedback for the generator
    return HumanMessage(content=res.content)


builder = MessageGraph()
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.set_entry_point("generate")


def should_continue(state: List[BaseMessage]):
    if len(state) > 2:
        # End after 3 iterations
        return END
    return "reflect"


builder.add_conditional_edges("generate", should_continue)
builder.add_edge("reflect", "generate")
graph = builder.compile()

In [8]:
async for event in graph.astream(
    [
        HumanMessage(
            content="Generate an essay on the topicality of The Little Prince and its message in modern life"
        )
    ],
):
    print(event)
    print("---")

{'generate': AIMessage(content='"The Little Prince" by Antoine de Saint-Exupéry is a timeless classic that continues to resonate with readers of all ages due to its profound messages and themes that are still relevant in modern life. The novella, first published in 1943, tells the story of a young prince who travels from planet to planet, meeting various characters and learning important life lessons along the way. The book\'s enduring topicality can be attributed to its exploration of universal themes such as love, friendship, loneliness, and the complexities of human nature.\n\nOne of the central messages of "The Little Prince" that remains pertinent in today\'s society is the importance of seeing beyond the surface and valuing what truly matters in life. In a world that often prioritizes material wealth and success, the little prince\'s interactions with characters like the vain rose and the greedy businessman serve as a poignant reminder of the emptiness of such pursuits. The princ

In [14]:
print(event)

{'generate': AIMessage(content='"The Little Prince" by Antoine de Saint-Exupéry continues to captivate readers with its timeless themes and messages that hold significant relevance in modern life. The novella\'s exploration of universal concepts such as love, friendship, loneliness, and the complexities of human nature resonates deeply with audiences of all ages. Through the little prince\'s interactions with various characters, the book imparts valuable lessons on the importance of looking beyond superficialities, nurturing meaningful connections, and embracing empathy and emotional intimacy in today\'s society.\n\nOne of the central themes of "The Little Prince" is the idea of seeing beyond the surface and valuing what truly matters in life. The little prince\'s encounters with characters like the vain rose and the greedy businessman highlight the emptiness of pursuing material wealth and superficial success. By emphasizing the significance of genuine human connection and cherishing 

In [9]:
ChatPromptTemplate.from_messages(event[END]).pretty_print()

KeyError: '__end__'